************Exercise -- 6 **********

What i have to do is:<br>
    Have to think fun project to complete it.<br>
    
As our previous model only generate english names<br>
let's genereate Neplease Names

In [ ]:
Where to get data from?
let's use gpt


In [51]:
import torch
words1 = open('nepali_names.txt','r').read().splitlines()

In [52]:
words = []
for w in words1:
    a,b = w.split()
    words.append(a.lower())
    words.append(b.lower())

In [53]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(chars)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [54]:
import torch.nn.functional as F

In [55]:
# create the dataset

xs, ys = [], []
for w in words:
  chs = ['.'] + list(w) + ['.']
  for ch1,ch2,ch3 in zip(chs,chs[1:],chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append((ix1*27)+ix2)
    ys.append(ix3)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

# initialize the 'network'
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27*27, 27), generator=g, requires_grad=True)

number of examples:  12443


In [56]:
from sklearn.model_selection import train_test_split

X_train,X_temp,y_train,y_temp = train_test_split(xs,ys,test_size = 0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp,y_temp, test_size=0.5, random_state = 42)

# X_train, y_train ma 80% data xa ani
#X_dev ra X_test ma 10 10% data xa ni

In [57]:
# print(X_train.size())
# print(X_dev.size())
# print(X_test.size())
train_size = len(X_train)
dev_size = len(X_dev)
test_size = len(y_test)
# X_train = X_train.float()
# y_train = y_train.float()
# X_dev = X_dev.float()
# y_dev = y_dev.float()
# y_train = y_train.float()
# y_test = y_test.float()

In [58]:
dev_loss = 1e10

In [59]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
import torch.nn as nn

criterion = nn.CrossEntropyLoss()

In [61]:
for k in range(1000):
    
    
    #forward pass for train-set
    
#     xenc = F.one_hot(X_train, num_classes = (27*27)).float() #input to the network: one-hot encoding
#     logits = no_of_examples * 27
    
    logits = W[X_train]  #log-counts predict gareko hai
    counts = logits.exp()  #coutns, equivalent to N
    probs = counts/counts.sum(1,keepdim = True) #probabilities for next character
    
    loss = criterion(logits,y_train) + 0.01 * (W**2).mean()  # i haven't understood this second part
    
    
    #forward pass for dev-set
    logits_dev = W[X_dev]
    counts_dev = logits_dev.exp()
    probs_dev = counts_dev/ counts_dev.sum(1, keepdim = True)
    loss_2 = criterion(logits_dev,y_dev)
    
    print(f"dev_loss: {loss_2.item()}")
    
    if(dev_loss < loss_2):
        print(f"early stopping done, stoped at: {k}")
        break
    else:
        dev_loss = loss_2
    #maile bujeko yo loss chai,
#     probs[torch.arrange(num),ys] this gives probability predcited by our model for ground truth. if it is less than 
#     100% model we give it a loss i.e -loglikelihood now model updates it's parameters.
    
#     Q. I am super-amazed how this loss worked? 
#     --> because we are not always sure what letters comes after pairs in real names. any letter can come. 
#         but some are more likely.
#     --> i think our model will twiks our W such that it gives more probabilty for more likely words based on training set
    
#     Q. Still confused how loss will converged? I suspect it will not. (noob me, think later)
#     Q. 

     
    print(f"loss: {loss.item()}")
    #backward pass:
    
    W.grad = None # paile sabaile gradient lai zero gareko
    loss.backward()  #magic hehe
    
    #update
    
    W.data += -50*W.grad   #remember to increment opp to direction of gradient, otherwise face the consequences)

    

dev_loss: 3.844693899154663
loss: 3.849792718887329
dev_loss: 3.2283008098602295
loss: 3.237870454788208
dev_loss: 2.7150275707244873
loss: 2.72585391998291
dev_loss: 2.3335764408111572
loss: 2.3396332263946533
dev_loss: 2.065885543823242
loss: 2.064405918121338
dev_loss: 1.8755134344100952
loss: 1.8681105375289917
dev_loss: 1.7315940856933594
loss: 1.7208688259124756
dev_loss: 1.6183726787567139
loss: 1.6066195964813232
dev_loss: 1.5278981924057007
loss: 1.5166445970535278
dev_loss: 1.4550189971923828
loss: 1.4450938701629639
dev_loss: 1.395776629447937
loss: 1.3875408172607422
dev_loss: 1.347025990486145
loss: 1.3405721187591553
dev_loss: 1.3063369989395142
loss: 1.301622748374939
dev_loss: 1.2718919515609741
loss: 1.268811821937561
dev_loss: 1.242353081703186
loss: 1.2407761812210083
dev_loss: 1.2167341709136963
loss: 1.2165236473083496
dev_loss: 1.1942964792251587
loss: 1.195321798324585
dev_loss: 1.1744751930236816
loss: 1.1766185760498047
dev_loss: 1.156828761100769
loss: 1.15998

In [62]:


xenc_test = F.one_hot(X_test, num_classes = (27*27)).float()
logits_test = xenc_test @ W
counts_test = logits_test.exp()
probs_test = counts_test/ counts_test.sum(1, keepdim = True)
loss_3 = -probs_test[torch.arange(test_size),y_test].log().mean()

print(loss_3.item())

0.8903403878211975


In [63]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix = 0
  gt = torch.randint(1,27,(1,)).item()  
  while True:

    # ----------
    # BEFORE:
    #p = P[ix]
    # ----------
    # NOW:
    ix = (ix*27) + (gt)
    out.append(itos[gt])
    xenc = F.one_hot(torch.tensor([ix]), num_classes=27*27).float()
    logits = xenc @ W # predict log-counts
    counts = logits.exp() # counts, equivalent to N
    p = counts / counts.sum(1, keepdims=True) # probabilities for next character
    # ----------

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    ix , gt = gt, ix
    
    if gt == 0:
      break
  print(''.join(out))

xouwjdedta
naka
bipesha
subina
najesh
utoysoash
anita
qumala
warabisha
sandbyainod


In [ ]:
#ok seems nice